### viewers setup

In [ ]:
from voxcellview.widget import VoxcellWidget

### loaders initialization

In [ ]:
from voxcell.core import CellCollection, VoxelData, Hierarchy
from brainbuilder.utils import bbp

cells = CellCollection()

# Brain Builder modules

##   Build.Region: Region of Interest

In [ ]:
from brainbuilder.select_region import select_region, select_hemisphere

### input parameters

In [ ]:
annotation = VoxelData.load_metaio("../data/P56_Mouse_annotation/annotation.mhd")
hierarchy = Hierarchy.load("../data/P56_Mouse_annotation/annotation_hierarchy.json")
full_density = VoxelData.load_metaio("../data/atlasVolume/atlasVolume.mhd")
region_name = "Primary somatosensory area"

### run module

In [ ]:
density = select_region(annotation.raw, full_density, hierarchy, region_name)
density.raw = select_hemisphere(density.raw)

### output

In [ ]:
VoxcellWidget().show_volume('density', density)

##  Build.Cells:  Cell Positions

In [ ]:
from brainbuilder.cell_positioning import cell_positioning

### input parameters

In [ ]:
total_cell_count = 200000

### run module

In [ ]:
cells.positions = cell_positioning(density, total_cell_count)

### output

In [ ]:
VoxcellWidget().show_points('position', cells)

## Build.EI:  E-I ratios

In [ ]:
from brainbuilder.assignment_synapse_class import assign_synapse_class_from_spatial_dist

### input parameters

In [ ]:
recipe_filename = "../data/bbp_recipe/builderRecipeAllPathways.xml"
recipe_sdist = bbp.load_recipe_as_spatial_distribution(recipe_filename, annotation, hierarchy, region_name)

### run module

In [ ]:
chosen_synapse_class = assign_synapse_class_from_spatial_dist(cells.positions, recipe_sdist)
cells.add_properties(chosen_synapse_class)

### output

In [ ]:
VoxcellWidget().show_property('synapse_class', cells)

## Build.Composition.ME: METype for Soma

In [ ]:
from brainbuilder.assignment_metype import assign_metype

### run module

In [ ]:
chosen_me = assign_metype(cells.positions, cells.properties.synapse_class, recipe_sdist)
cells.add_properties(chosen_me)

### output

#### mtypes

In [ ]:
VoxcellWidget().show_property('mtype', cells)

#### etypes

In [ ]:
VoxcellWidget().show_property('etype', cells)

## Build.Placement: Morphology assignment

In [ ]:
from brainbuilder.assignment_morphology import assign_morphology

### input parameters

In [ ]:
neurondb_filename = "../data/bbp_recipe/neurondb.dat"
neuron_sdist = bbp.load_neurondb_v4_as_spatial_distribution(neurondb_filename, annotation, hierarchy, region_name, percentile=0.92)

### run module

In [ ]:
chosen_morphology = assign_morphology(cells.positions, cells.properties[['mtype', 'etype']], neuron_sdist)
cells.add_properties(chosen_morphology)

### output

In [ ]:
VoxcellWidget().show_property('morphology', cells)

## Orientation assignement

In [ ]:
from brainbuilder.orientation_field_sscx import compute_orientation_field
from brainbuilder.assignment_orientation import assign_orientations

### run module

In [ ]:
orientation_field = compute_orientation_field(annotation, hierarchy, region_name)
cells.orientations = assign_orientations(cells.positions, orientation_field)

### output

In [ ]:
#notebook.show_placement('placement', cells)